# Lista de Exercícios 04

###### Acadêmicos: Gabriel Schneider de Jesus e Luciane Tedesco

Este trabalho visa demonstrar uma aplicação de upload e download de arquivos com a vulnerabilidade "Path Traversal", na qual é possível obter acesso não autorizado de arquivos e diretórios.

## Questão 1
Demostração da vunerabilidade "Path Traversal".

#### Aplicando Path Traversal

In [ ]:
import os

from flask import Flask, redirect, request, send_file, abort, flash

app = Flask(__name__)
SERVER_FOLDER = 'server'


@app.route("/", methods=['GET'])
def get_file():
    file_name = request.args.get("file")

    if not file_name:
        abort(404)
    return send_file(os.path.join(SERVER_FOLDER, file_name))


@app.route("/", methods=['POST'])
def send_file():
    if 'file' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('No selected file')
        return redirect(request.url)

    dest_file = request.form['name']

    file.save(os.path.join(SERVER_FOLDER, dest_file))
    return {'success': True}, 200, {'ContentType': 'application/json'}


if __name__ == '__main__':
    app.run(debug=True)

#### Exemplo de saída (upload) com vunerabilidade

<img src="Images/ExemploUploadProblematico.png" width = 950>

No exemplo acima, podemos observar que é possível navegar na estrutura de pastas para acessar arquivos que não deveriam ser acessíveis a partir da aplicação.

O método GET representa o download de arquivos, onde o usuário informa apenas o nome do arquivo presente no diretório "server". O arquivo é aberto para visualização.
O método POST representa o upload de arquivos, onde o usuário deveria selecionar um arquivo do SO e informar um nome. O arquivo é salvo no diretório "server" com o nome escolhido pelo usuário.

Porém o usuário pode causar a vulnerabilidade, informando no campo nome um diretório diferente de "server".

Uso esperado do download: 
```rest
localhost:5000/?file=file01
``` 
Uso indevido do download: 
```rest
localhost:5000/?file=/home/luciane/git/SistemasSeguros/Exercicio04/temp/file10
```

Uso esperado do upload: 
```rest
localhost:5000/

Body
name:file02
file: Teste
``` 
Uso indevido do upload: 
```rest
localhost:5000/

Body
name:/home/luciane/git/SistemasSeguros/Exercicio04/tmp/file10
file: Teste
```


## Questão 2
Utilização da técnica de atribuição de códigos aos arquivos para impedir que o usuário explore a vulnerabilidade "Path Traversal".

In [ ]:
import os

from flask import Flask, redirect, request, send_file, abort, flash

app = Flask(__name__)
SERVER_FOLDER = 'server'

files = {
    "01": "file01",
    "02": "file02",
}


def get_dictionary(file_id):
    return files[file_id]


@app.route("/", methods=['GET'])
def get_file():
    file_name = request.args.get("file")

    if not file_name:
        abort(404)
    return send_file(os.path.join(SERVER_FOLDER, get_dictionary(file_name)))


@app.route("/", methods=['POST'])
def send_file():
    if 'file' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        flash('No selected file')
        return redirect(request.url)

    file_name = request.form['name']

    file.save(os.path.join(SERVER_FOLDER, get_dictionary(file_name)))
    return {'success': True}, 200, {'ContentType': 'application/json'}


if __name__ == '__main__':
    app.run(debug=True)


Para corrigir a vulnerabilidade, definimos um identificador único para cada arquivo, com isso o usuário nao poderá mais chamar o arquivo diretamente.

Uso do download: 
```rest
localhost:5000/?file=01
``` 

Uso do upload: 
```rest
localhost:5000/

Body
name:01
file: Teste
``` 

#### Exemplo de saida (upload) sem vunerabilidade

<img src="Images/UploadConcertado.png" width = 950> 